In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import scale
from tqdm import tqdm
from surprise import SVD
import pickle

In [13]:
df_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\track_artists.csv"
train_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\train"
test_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\test"
pred_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\ensemble_bpr_artist_svd_625_075_ss"
bpr_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\final_bpr_model_backup"
svd_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\final_svd_model_backup"
rec_bpr_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\final_bpr_rec_625_backup"
pred_binary_path = r"C:\Users\dlbol\Downloads\likes\likes\likes_data\ensemble_bpr_artist_svd_625_075_ss_binary"

In [3]:
%%time

rec = pickle.load(open(rec_bpr_path, "rb"))

CPU times: total: 844 ms
Wall time: 1.04 s


In [4]:
rec.shape

(2, 289914, 625)

In [5]:
%%time
df = pd.read_csv(df_path)
df.index = df['trackId']

CPU times: total: 78.1 ms
Wall time: 58 ms


In [6]:
%%time

algo = pickle.load(open(svd_path, "rb"))

CPU times: total: 29.5 s
Wall time: 31.6 s


N: 625, norm_all: False, preproc_model: standart, preproc_algo: standart, coef_art: 0.75

In [7]:
%%time

model_score = scale(rec[1], with_mean=True, with_std=True, axis=1)

CPU times: total: 6.16 s
Wall time: 6.74 s


In [8]:
first_id_test = 1160084
last_id_test = 1449997
N = 625
coef_art = 0.75

In [9]:
%%time

result = []
user_id = first_id_test
range_shape = rec[0].shape[0]
for i in tqdm(range(range_shape)):
    df_final = pd.DataFrame({'trak_id': rec[0][i], 'model_score': model_score[i]})
    df_final = pd.merge(df_final, df, how="inner", left_on='trak_id', right_on=df.index, sort=False)
    df_final['artist_score'] = np.array([algo.predict(uid=user_id, iid=iid).est for iid in df_final.artistId.values])
    df_final['artist_score'] = scale(df_final.artist_score.values, with_mean=True, with_std=True, axis=0)
    df_final['final_score'] = df_final.model_score + df_final.artist_score * coef_art
    final_rec = df_final.sort_values(by='final_score', ascending=False).trak_id.values[:100]
    result.append(' '.join(map(str, final_rec)) + '\n')
    user_id += 1

  0%|                                                                           | 160/289914 [00:09<4:03:10, 19.86it/s]c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
  4%|██▉                                                                      | 11677/289914 [10:39<3:54:36, 19.77it/s]c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
 14%|██████████                                                               | 39855/289914 [34:53<3:31:41, 19.69it/s]c:\python38\lib\site-packages\sklearn\preprocessing\_data.py:258: UserWarning: Numerical issues were encountered when scaling the data and might not be solved.

CPU times: total: 4h 25min 31s
Wall time: 4h 25min 59s


In [10]:
%%time

with open(pred_path, 'w') as f:
    f.writelines(result)

CPU times: total: 1.16 s
Wall time: 1.18 s


In [12]:
len(result)

289914

In [14]:
pickle.dump(result, open(pred_binary_path, "wb"))